In [47]:
import pandas as pd

from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
from tqdm import tqdm

In [48]:
df_train = pd.read_csv('../input/train_extra_feat.csv')
df_train.head()

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,...,mention_count,char_count,punctuation_count,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count
0,1,no%20keyword,no%20location,Our Deeds are the Reason of this #earthquake M...,1,13,13,6,7,0,...,0,69,1,1,4.384615,69.0,0,Our Deeds are the Reason of this earthquake Ma...,0,0
1,4,no%20keyword,no%20location,Forest fire near La Ronge Sask. Canada,1,7,7,0,7,0,...,0,38,1,2,4.571429,18.5,0,Forest fire near La Ronge Sask. Canada,2,0
2,5,no%20keyword,no%20location,All residents asked to 'shelter in place' are ...,1,22,20,11,11,0,...,0,133,3,2,5.090909,66.0,0,All residents asked to ishelter in place' are ...,3,0
3,6,no%20keyword,no%20location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,7,0,...,0,65,2,1,7.125000,64.0,0,"13,000 people receive wildfires evacuation ord...",2,0
4,7,no%20keyword,no%20location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,7,9,0,...,0,88,2,1,4.500000,87.0,0,Just got sent this photo from Ruby Alaska as s...,1,0


In [74]:
df_test = pd.read_csv('../input/test_extra_feat.csv')
df_test.head()

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,hashtag_count,...,char_count,punctuation_count,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count,emoji_count
0,0,no%20keyword,no%20location,Just happened a terrible car crash,6,6,2,4,0,0,...,34,0,1,4.833333,34.0,0,Just happened a terrible car crash,0,0,0
1,2,no%20keyword,no%20location,"Heard about #earthquake is different cities, s...",9,9,2,7,0,1,...,64,3,1,6.222222,64.0,0,"Heard about earthquake is different cities, st...",2,0,0
2,3,no%20keyword,no%20location,"there is a forest fire at spot pond, geese are...",19,19,10,9,0,0,...,96,2,1,4.105263,96.0,0,"there is a forest fire at spot pond, geese are...",3,0,0
3,9,no%20keyword,no%20location,Apocalypse lighting. #Spokane #wildfires,4,4,0,4,0,2,...,40,3,2,9.250000,19.5,0,Apocalypse lighting. Spokane wildfires,3,0,0
4,11,no%20keyword,no%20location,Typhoon Soudelor kills 28 in China and Taiwan,8,8,2,6,0,0,...,45,0,1,4.750000,45.0,1,Typhoon Soudelor kills 28 in China and Taiwan,1,0,0


In [49]:
client = language.LanguageServiceClient()

In [58]:
all_scores = []
all_magnitudes = []
all_text = df_train['clean_text'].values
with tqdm(total=len(all_text)) as pbar:
    for tweet in all_text:
        document = types.Document(content=tweet, type=enums.Document.Type.PLAIN_TEXT)
        try:
            annotations = client.analyze_sentiment(document=document)
            sentiment = annotations.sentences[0].sentiment
            all_scores.append(sentiment.score)
            all_magnitudes.append(sentiment.magnitude)
        except:
            print(tweet)
            all_scores.append('')
            all_magnitudes.append('')
        pbar.update(1)

 22%|██▏       | 1707/7613 [05:24<16:27,  5.98it/s] 

When love and hate collided part II Lanjut dirumah. yagitudeh Jake at Rumah Cipinang 


 82%|████████▏ | 6267/7613 [19:11<03:45,  5.98it/s]

kesabaran membuahkan hasil indah pada saat tepat! life isn not about waiting for the storm to pass it is about learning to dance in the rain.


100%|██████████| 7613/7613 [23:14<00:00,  5.46it/s]


In [63]:
[(index, score) for index, score in enumerate(all_scores) if score == '']

[(1705, ''), (6265, '')]

In [65]:
df_train['clean_text'].values[1705]

'When love and hate collided part II Lanjut dirumah. yagitudeh Jake at Rumah Cipinang '

In [66]:
document = types.Document(content='When love and hate collided', type=enums.Document.Type.PLAIN_TEXT)
annotations = client.analyze_sentiment(document=document)
sentiment = annotations.sentences[0].sentiment
all_scores[1705] = sentiment.score
all_magnitudes[1705] = sentiment.magnitude

In [67]:
[(index, score) for index, score in enumerate(all_scores) if score == '']

[(6265, '')]

In [68]:
df_train['clean_text'].values[6265]

'kesabaran membuahkan hasil indah pada saat tepat! life isn not about waiting for the storm to pass it is about learning to dance in the rain.'

In [69]:
document = types.Document(content='life isn not about waiting for the storm to pass it is about learning to dance in the rain.', 
                          type=enums.Document.Type.PLAIN_TEXT)
annotations = client.analyze_sentiment(document=document)
sentiment = annotations.sentences[0].sentiment
all_scores[6265] = sentiment.score
all_magnitudes[6265] = sentiment.magnitude

In [70]:
[(index, score) for index, score in enumerate(all_scores) if score == '']

[]

In [71]:
df_train['sentiment_score'] = all_scores
df_train['sentiment_magnitude'] = all_magnitudes
df_train.head()

,id,keyword,location,text,target,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,...,punctuation_count,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count,sentiment_score,sentiment_magnitude
0,1,no%20keyword,no%20location,Our Deeds are the Reason of this #earthquake M...,1,13,13,6,7,0,...,1,1,4.384615,69.0,0,Our Deeds are the Reason of this earthquake Ma...,0,0,0.6,0.6
1,4,no%20keyword,no%20location,Forest fire near La Ronge Sask. Canada,1,7,7,0,7,0,...,1,2,4.571429,18.5,0,Forest fire near La Ronge Sask. Canada,2,0,0.0,0.0
2,5,no%20keyword,no%20location,All residents asked to 'shelter in place' are ...,1,22,20,11,11,0,...,3,2,5.090909,66.0,0,All residents asked to ishelter in place' are ...,3,0,0.0,0.0
3,6,no%20keyword,no%20location,"13,000 people receive #wildfires evacuation or...",1,8,8,1,7,0,...,2,1,7.125000,64.0,0,"13,000 people receive wildfires evacuation ord...",2,0,0.0,0.0
4,7,no%20keyword,no%20location,Just got sent this photo from Ruby #Alaska as ...,1,16,15,7,9,0,...,2,1,4.500000,87.0,0,Just got sent this photo from Ruby Alaska as s...,1,0,0.0,0.0


In [72]:
df_train.to_csv('../input/train_extra_feat_sentiment.csv', index=False)

In [75]:
all_scores = []
all_magnitudes = []
all_text = df_test['clean_text'].values
with tqdm(total=len(all_text)) as pbar:
    for index, tweet in enumerate(all_text):
        document = types.Document(content=tweet, type=enums.Document.Type.PLAIN_TEXT)
        try:
            annotations = client.analyze_sentiment(document=document)
            sentiment = annotations.sentences[0].sentiment
            all_scores.append(sentiment.score)
            all_magnitudes.append(sentiment.magnitude)
        except:
            print(index, tweet)
            all_scores.append('')
            all_magnitudes.append('')
        pbar.update(1)

 37%|███▋      | 1212/3263 [04:01<05:39,  6.04it/s]

1210 Devastation Walkthrough Part 11 przez YouTube


100%|██████████| 3263/3263 [10:11<00:00,  5.34it/s]


In [76]:
[(index, score) for index, score in enumerate(all_scores) if score == '']

[(1210, '')]

In [77]:
document = types.Document(content='Devastation Walkthrough Part 11', 
                          type=enums.Document.Type.PLAIN_TEXT)
annotations = client.analyze_sentiment(document=document)
sentiment = annotations.sentences[0].sentiment
all_scores[1210] = sentiment.score
all_magnitudes[1210] = sentiment.magnitude

In [78]:
[(index, score) for index, score in enumerate(all_scores) if score == '']

[]

In [79]:
df_test['sentiment_score'] = all_scores
df_test['sentiment_magnitude'] = all_magnitudes
df_test.head()

,id,keyword,location,text,word_count,unique_word_count,stop_word_count,non_stop_word_count,url_count,hashtag_count,...,sentence_count,mean_word_length,mean_sentence_length,number_count,clean_text,typo_count,slang_count,emoji_count,sentiment_score,sentiment_magnitude
0,0,no%20keyword,no%20location,Just happened a terrible car crash,6,6,2,4,0,0,...,1,4.833333,34.0,0,Just happened a terrible car crash,0,0,0,-0.9,0.9
1,2,no%20keyword,no%20location,"Heard about #earthquake is different cities, s...",9,9,2,7,0,1,...,1,6.222222,64.0,0,"Heard about earthquake is different cities, st...",2,0,0,0.7,0.7
2,3,no%20keyword,no%20location,"there is a forest fire at spot pond, geese are...",19,19,10,9,0,0,...,1,4.105263,96.0,0,"there is a forest fire at spot pond, geese are...",3,0,0,-0.6,0.6
3,9,no%20keyword,no%20location,Apocalypse lighting. #Spokane #wildfires,4,4,0,4,0,2,...,2,9.250000,19.5,0,Apocalypse lighting. Spokane wildfires,3,0,0,0.4,0.4
4,11,no%20keyword,no%20location,Typhoon Soudelor kills 28 in China and Taiwan,8,8,2,6,0,0,...,1,4.750000,45.0,1,Typhoon Soudelor kills 28 in China and Taiwan,1,0,0,0.1,0.1


In [80]:
df_test.to_csv('../input/test_extra_feat_sentiment.csv', index=False)